In [2]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import random
import math

# helps in text preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# helps in model building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.layers import Bidirectional, GlobalMaxPool1D

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pickle

#read Religion topic
religion_path = "C:/Users/steven/Documents/spam/Topic_00.en"
#save each sentence into np.array
with open(religion_path, encoding="utf8") as f:
    relig_content = [line.rstrip() for line in f]
rel_arr = np.array(relig_content)
#add for each sentence the same class (e.g.: 0)
rel_arr_label = np.full(len(rel_arr), 0)

import random
import math
random.seed(42)
#read all other topics
other_topic_numb = 9
max_topic_numb = 9
general_arr = []
count = 0
segments_pro_topic = math.ceil(len(rel_arr) / other_topic_numb)
while count <= max_topic_numb:
    if(count != 0):#to skip undesired topics
        general_path = f"C:/Users/steven/Documents/spam/Topic_{count:02d}.en"
        #save each sentence into np.array
        #print(segments_pro_topic)
        #save some sentences into np.array (the same amount as Religion topic ones)
        with open(general_path, encoding="utf8") as f:
            general_arr = general_arr + (random.sample([line.rstrip() for line in f], segments_pro_topic))
    count = count + 1

#print(len(general_arr))
random.shuffle(general_arr)
#add for each sentence the same class (Different from Religion! e.g.: 1)
general_arr_label = np.full(len(general_arr), 1)

conc_arr = np.concatenate((rel_arr,general_arr))
conc_arr_label = np.concatenate((rel_arr_label,general_arr_label))

x_train, x_test, y_train, y_test = train_test_split(conc_arr, conc_arr_label, test_size=0.20, random_state=42)

In [3]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(x_train)

print(len(t.word_index))
word_index = t.word_index
encoded_train = t.texts_to_sequences(x_train)
encoded_test = t.texts_to_sequences(x_test)

length=[]
for conc in conc_arr:
    #print(len(conc))
    split = conc.split(" ")
    #print(len(test1))
    length.append(len(split))
max_value = max(length)
print('Maximum value:', max_value)

max_length = max_value

padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')
#print(len(padded_train))

#word_index = dict(zip((o for o in x_train), range(len(x_train))))
#print(word_index)
#print(len(encoded_train))


102234
Maximum value: 200


In [6]:
#make a dict mapping words (strings) to their NumPy vector representation:
import pandas as pd
import numpy as np

path_to_glove_file = "C:/Users/Steven/Downloads/glove.6B.100d.txt/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file, encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index)) 

Found 400000 word vectors.


In [7]:
"""
NP matrix for the "embedding" layer in Keras. To map index of pretrained vectos with the word of index i in the vectorizer vocab
"""

num_tokens = len(t.word_index) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))



Converted 62670 words (39564 misses)


In [8]:
embeddings_index.get("allah")

array([ 0.34648  ,  0.2096   ,  0.070741 , -0.97231  , -0.052933 ,
        0.65437  , -0.86157  , -0.69758  ,  0.25109  , -0.32881  ,
        0.81759  , -0.34523  ,  0.53054  ,  0.068022 ,  0.20367  ,
       -0.96712  ,  0.6745   , -0.050067 , -1.6481   ,  0.86177  ,
        0.11933  , -0.96415  , -0.84824  ,  0.029813 ,  0.29883  ,
        1.2351   , -0.62322  , -0.0096935,  0.57381  ,  0.26418  ,
        0.35808  ,  0.040504 ,  0.13176  ,  0.19399  ,  0.11848  ,
        0.31298  , -0.22346  ,  0.29079  ,  0.35653  , -0.22875  ,
       -0.65771  ,  1.1989   ,  0.26678  , -0.47767  ,  0.10477  ,
       -0.58318  ,  0.1232   ,  0.14058  , -0.12584  , -0.50352  ,
       -0.58613  ,  0.82644  ,  1.1736   ,  0.78827  , -1.0276   ,
        0.3125   ,  0.81627  , -0.13461  ,  0.02046  , -0.50721  ,
        0.31498  ,  0.077944 , -0.66653  , -0.43601  ,  0.040687 ,
        0.69847  ,  0.36838  , -0.30218  , -0.37972  , -0.37998  ,
        0.81017  ,  0.26011  ,  1.0392   , -1.5402   ,  0.5663

In [9]:
"""

Load the pretrained Glove embeddings to the embedding layer in keras. Set Trainable False.
"""
import keras
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    input_length=max_length
)


In [10]:
#Build a model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.layers import Bidirectional, GlobalMaxPool1D
n_lstm = 20
drop_lstm =0.2
embed_size = 100 # how big is each word vector
maxlen = 2000 # max number of words in a question to use
max_features = 44971


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
model = Sequential([
    embedding_layer,
    Bidirectional(LSTM(n_lstm, dropout=drop_lstm, input_length=max_length,  return_sequences=True)), 
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 100)          10223600  
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 40)           19360     
_________________________________________________________________
dense (Dense)                (None, 200, 128)          5248      
_________________________________________________________________
dense_1 (Dense)              (None, 200, 1)            129       
Total params: 10,248,337
Trainable params: 24,737
Non-trainable params: 10,223,600
_________________________________________________________________


In [12]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

def c_report(y_true, y_pred):
   print("Classification Report")
   print(classification_report(y_true, y_pred))
   acc_sc = accuracy_score(y_true, y_pred)
   print("Accuracy : "+ str(acc_sc))
   return acc_sc

def plot_confusion_matrix(y_true, y_pred):
   mtx = confusion_matrix(y_true, y_pred)
   sns.heatmap(mtx, annot=True, fmt='d', linewidths=.5, 
               cmap="Blues", cbar=False)
   plt.ylabel('True label')
   plt.xlabel('Predicted label')

In [14]:
#test with BiLSTM model
model.fit(padded_train, y_train, batch_size=128, epochs=5, validation_data=(padded_test, y_test), callbacks=[early_stop])


Epoch 1/5
1833/1833 [==============================] - 692s 375ms/step - loss: 0.2859 - accuracy: 0.8807 - val_loss: 0.2203 - val_accuracy: 0.9123
Epoch 2/5
1833/1833 [==============================] - 732s 400ms/step - loss: 0.2250 - accuracy: 0.9068 - val_loss: 0.2299 - val_accuracy: 0.9014
Epoch 3/5
1833/1833 [==============================] - 750s 409ms/step - loss: 0.2108 - accuracy: 0.9128 - val_loss: 0.1989 - val_accuracy: 0.9192
Epoch 4/5
1833/1833 [==============================] - 620s 338ms/step - loss: 0.2028 - accuracy: 0.9167 - val_loss: 0.1859 - val_accuracy: 0.9248
Epoch 5/5
1833/1833 [==============================] - 621s 339ms/step - loss: 0.1983 - accuracy: 0.9180 - val_loss: 0.1746 - val_accuracy: 0.9282


In [15]:
#test BILSTM
preds = (model.predict(padded_test) > 0.1).astype("int32")
#second option for prediction based on the results o to 1
#y_predict  = [1 if o>0.5 else 0 for o in model.predict(x_test_features)]

In [28]:
#saving and loading the model.
model.save("spam_model_1\spam_model_3")


#load model
s_model = tf.keras.models.load_model("spam_model_4")


INFO:tensorflow:Assets written to: spam_model_1\spam_model_3\assets


INFO:tensorflow:Assets written to: spam_model_1\spam_model_3\assets


In [19]:
model.save('spam_model_1\my_model_3.h5')

In [22]:
preds2 = model.evaluate(padded_test, y_test, verbose= False)
print(f'Test- Results - Loss:{preds2[0]} - Accuracy:{100*preds2[1]}%')

Test- Results - Loss:0.174625426530838 - Accuracy:92.82114505767822%


In [32]:
# Defining prediction function option 1
print(s_model.summary())
#Change max_len according to the shape of the input/embedding layer
def predict_spam(predict_msg):
    with open(predict_msg, encoding="utf8") as f:
        test_content = [line.rstrip() for line in f]
    test_arr = np.array(test_content)
    t = Tokenizer()
    max_len = 20
    test_proc = t.texts_to_sequences((o)for o in test_arr)
    test_proc = pad_sequences(test_proc, maxlen=max_len, padding='post')
    #print(test_proc.shape)
    pred = (s_model.predict(test_proc) > 0.1).astype("int32")
    return pred
    index, j = np.where(pred == 0)
    
    print(index)
    print(j)
    print(len(index))
    print(len(j))
    selection = [test_arr[ind] for ind in index]
    print(selection[:10])
    
predict_spam("C:/Users/steven/Documents/spam/Topic_02.en")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 24)            2453640   
_________________________________________________________________
flatten (Flatten)            (None, 480)               0         
_________________________________________________________________
dense (Dense)                (None, 500)               240500    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               100200    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])